In [1]:
from pyfuseki import FusekiUpdate, FusekiQuery
import httpx
from rdflib import Graph
from pysolr import Solr
from api.src.schemas.bibframe.work import Work_Schema, Work_Edit, Work_Response

In [14]:
acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')
prefix = """PREFIX work: <https://bibliokeia.com/resources/work/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>"""

In [12]:
query = prefix+"""SELECT * WHERE { 
  graph work:bk-1 {
  work:bk-1 ?p ?o } }"""

response = acervoQuery.run_sparql(query)
# response = response.convert()
# bindings = response['results']['bindings']
# bindings

Type

In [29]:
queryType = prefix+"""SELECT * WHERE { 
  graph work:bk-1 {
  work:bk-1 rdf:type ?o } }"""
response = acervoQuery.run_sparql(queryType)
response = response.convert()
bindings = response['results']['bindings']

{'head': {'vars': ['o']},
 'results': {'bindings': [{'o': {'type': 'uri',
     'value': 'http://id.loc.gov/ontologies/bibframe/Work'}},
   {'o': {'type': 'uri',
     'value': 'http://id.loc.gov/ontologies/bibframe/Text'}}]}}

In [36]:
values = list()
for i in bindings:
    value = i['o']['value'].split("/")[-1]
    values.append(value)
print(values)

['Work', 'Text']


In [11]:
for i in bindings:
    metadado = i['p']['value']
    print(i['p']['value'])

http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://id.loc.gov/ontologies/bibframe/subject
http://id.loc.gov/ontologies/bibframe/subject
http://id.loc.gov/ontologies/bibframe/subject
http://id.loc.gov/ontologies/bibframe/contribution
http://id.loc.gov/ontologies/bibframe/language
http://id.loc.gov/ontologies/bibframe/adminMetadata
http://id.loc.gov/ontologies/bibframe/title
http://id.loc.gov/ontologies/bibframe/classification


Classification

In [37]:
query = """SELECT ?p ?o WHERE { graph work:bk-1 {
  work:bk-1  bf:classification ?classification .
    ?classification ?p ?o 
  } }"""

queryClassfication = prefix+query
response = acervoQuery.run_sparql(queryClassfication)
response = response.convert()
bindings = response['results']['bindings']
bindings

[{'p': {'type': 'uri',
   'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
  'o': {'type': 'uri',
   'value': 'http://id.loc.gov/ontologies/bibframe/ClassificationDdc'}},
 {'p': {'type': 'uri',
   'value': 'http://id.loc.gov/ontologies/bibframe/classificationPortion'},
  'o': {'type': 'literal',
   'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
   'value': '584.6'}},
 {'p': {'type': 'uri',
   'value': 'http://id.loc.gov/ontologies/bibframe/itemPortion'},
  'o': {'type': 'literal', 'value': 'O48c'}}]

In [49]:
classification = {}
for i in bindings:
    metadadoUri = i['p']['value']
    if metadadoUri == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type':
        value = i['o']['value'].split('/')[-1]
        classification['type'] = value
    else:
        metadado = i['p']['value'].split('/')[-1]
        value = i['o']['value'].split('/')[-1]
        classification[metadado] = value

        print(metadado)

print(classification)

classificationPortion
itemPortion
{'type': 'ClassificationDdc', 'classificationPortion': '584.6', 'itemPortion': 'O48c'}


contribution

In [50]:
query = """SELECT ?p ?o WHERE { graph work:bk-1 {
  work:bk-1  bf:contribution ?contribution .
    ?contribution ?p ?o 
  } }"""

queryContribution = prefix+query
response = acervoQuery.run_sparql(queryContribution)
response = response.convert()
bindings = response['results']['bindings']
bindings

[{'p': {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#label'},
  'o': {'type': 'literal', 'value': 'Karl R. Popper MUDADO'}},
 {'p': {'type': 'uri',
   'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
  'o': {'type': 'uri',
   'value': 'http://id.loc.gov/ontologies/bibframe/Contribution'}},
 {'p': {'type': 'uri', 'value': 'http://id.loc.gov/ontologies/bibframe/role'},
  'o': {'type': 'uri', 'value': 'http://id.loc.gov/vocabulary/relators/aut'}},
 {'p': {'type': 'uri', 'value': 'http://id.loc.gov/ontologies/bibframe/agent'},
  'o': {'type': 'uri',
   'value': 'https://bibliokeia.com/authorities/names/n80032184'}}]

In [73]:
contribution = {}
for i in bindings:
    metadadoUri = i['p']['value']
    if metadadoUri == 'http://www.w3.org/2000/01/rdf-schema#label':
        contribution['label'] = i['o']['value']
    elif metadadoUri == 'http://id.loc.gov/ontologies/bibframe/role':
        value = i['o']['value'].split('/')[-1]
        contribution['role'] = value
    elif metadadoUri == 'http://id.loc.gov/ontologies/bibframe/agent':
        value = i['o']['value']
        contribution['uri'] = value
        
print(contribution)

{'label': 'Karl R. Popper MUDADO', 'role': 'aut', 'uri': 'https://bibliokeia.com/authorities/names/n80032184'}


In [ ]:
x = {
  "type": [
    "Work",
    "Text"
  ],
  "classification": {
    "type": "ClassificationDdc",
    "classificationPortion": "584.6",
    "itemPortion": "O48c"
  },
  "contribution": {
    "label": "Karl R. Popper MUDADO",
    "role": "aut",
    "uri": "https://bibliokeia.com/authorities/names/n80032184"
  }
}

r = Work_Response(**x)
r



LANGUAGEN

In [4]:
bkID = 'bk-1'
query = "SELECT * WHERE { graph work:"+bkID+""" {
            work:"""+bkID+"""  bf:language ?language 
            } }"""
queryLanguage = prefix+query
response = acervoQuery.run_sparql(queryLanguage)
response = response.convert()
bindings = response['results']['bindings']
bindings

[{'language': {'type': 'uri',
   'value': 'http://id.loc.gov/vocabulary/languages/eng'}}]

In [8]:
for i in bindings:
    code = i['language']['value'].split("/")[-1]
    print(code)

eng


SUBJECT

In [21]:
authorityQuery = FusekiQuery('http://localhost:3030', 'authorities')
authorityUp = FusekiUpdate('http://localhost:3030', 'authorities')

In [18]:
bkID = "bk-2"
query = "SELECT * WHERE { graph work:"+bkID+""" {
  work:"""+bkID+"""  bf:subject ?subject } }"""



In [19]:
prefix  = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX work: <https://bibliokeia.com/resources/work/>
PREFIX subjects: <https://bibliokeia.com/authorities/subjects/>
PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>"""

querySubject = prefix+query
response = acervoQuery.run_sparql(querySubject)
response = response.convert()
bindings = response['results']['bindings']
bindings


[{'subject': {'type': 'uri',
   'value': 'https://bibliokeia.com/authorities/subjects/n80002329'}}]

In [22]:
subjects = list()
for i in bindings:
    subjectID = i['subject']['value'].split("/")[-1]
    query = """SELECT * WHERE { graph subjects:"""+subjectID+""" {
        subjects:"""+subjectID+""" madsrdf:authoritativeLabel ?label .
        subjects:"""+subjectID+""" rdf:type ?type .
        FILTER(?type != madsrdf:Authority)
    } }"""
    querySubject = prefix+query
    response = authorityQuery.run_sparql(querySubject)
    response = response.convert()
    [response] = response['results']['bindings']
    label = response['label']['value']
    tipo = response['type']['value'].split("#")[-1]
    s = {"label": label, "type": tipo,
            "uri": f"https://bibliokeia.com/authorities/subjects/{subjectID}"}

    subjects.append(s)
subjects

ValueError: not enough values to unpack (expected 1, got 0)

In [24]:
for i in bindings:
    subjectID = i['subject']['value'].split("/")[-1]
    print(subjectID)
    

n80002329


In [49]:
for [i] in subjects:
    label = i['label']['value']
    tipo = i['type']['value'].split("#")[-1]
    s = {'label': label, 'type': tipo}
    print(s)

{'label': 'Metodologia', 'type': 'Topic'}


In [37]:
subjects[0][0]

{'label': {'type': 'literal', 'xml:lang': 'pt', 'value': 'Metodologia'},
 'type': {'type': 'uri', 'value': 'http://www.loc.gov/mads/rdf/v1#Topic'}}

In [17]:
d = """DELETE { graph subjects:n80002329 { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""
deleteSubject = prefix+d

response = authorityUp.run_sparql(deleteSubject)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [19]:
edit = {
  "subjects": [
    {
      "label": "Metodologia",
      "uri": "https://bibliokeia.com/authorities/subjects/sh85084414"
    }
  ]
}
obj = Work_Edit(**edit)
obj

Work_Edit(ID=None, content=None, mainTitle=None, subtitle=None, variantTitle=None, primaryContributionAgent=None, primaryContributionUri=None, primaryContributionRole=None, primaryContributionRoleUri=None, subjects=[Subject(label='Metodologia', uri='https://bibliokeia.com/authorities/subjects/sh85084414')], language=None, languageCode=None, cdd=None, cutter=None, serie=None, serieURI=None)

In [29]:
doc = {"id": "bk-i"}
for k, v in obj:
    if v:
            if k == 'subjects':
                  subs = [i.label for i in v]
                  doc[k] = {"set": subs}
                    

In [30]:
doc

{'id': 'bk-i', 'subjects': {'set': ['Metodologia']}}

In [2]:
work = {
  "ID": "bk-1",
  "content": "Texto",
  "title": {
    "mainTitle": "Conjecturas e refutações"
  },
  "primaryContribution": {
    "label": "Karl R. Popper",
    "role": "aut",
    "uri": "https://bibliokeia.com/authorities/names/n80032184"
  },
  "subjects": [
    {
      "label": "Metodologia",
      "type": "Topic",
      "uri": "https://bibliokeia.com/authorities/subjects/sh85084414"
    }
  ],
  "language": "Portugês",
  "languageCode": "por",
  "cdd": "542.3",
  "cutter": "O48c"
}

request = Work_Schema(**work)
request

Work_Schema(ID='bk-1', content='work', title=Title(mainTitle='Conjecturas e refutações', subtitle=None, variantTitle=None), primaryContribution=Contribution(label='Karl R. Popper', role='aut', uri='https://bibliokeia.com/authorities/names/n80032184'), subjects=[Subject(label='Ciência', type='Topic', uri='https://bibliokeia.com/authorities/subjects/n80002329')], language='Portugês', languageCode='por', cdd='542.3', cutter='O48c', serie=None, serieURI=None)

In [5]:
request.primaryContribution.label

Contribution(label='Karl R. Popper', role='aut', uri='https://bibliokeia.com/authorities/names/n80032184')

In [6]:
request.primaryContribution.label

'Karl R. Popper'

In [11]:
request.subjects

[Subject(label='Ciência', type='Topic', uri='https://bibliokeia.com/authorities/subjects/n80002329')]

In [12]:
for subject in request.subjects:
    print(subject)

label='Ciência' type='Topic' uri='https://bibliokeia.com/authorities/subjects/n80002329'


DELETE WORK

In [27]:
upAcervo = FusekiUpdate('http://localhost:3030', 'acervo')

d = """DELETE { graph <https://bibliokeia.com/resources/work/bkFHJFFF-1> { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = upAcervo.run_sparql(d)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}